<a href="https://colab.research.google.com/github/ttornike1991/Deep-Learning-Colab-projects/blob/main/PROJECTTOMATOES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [12]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [18]:
#import libraries
from tensorflow import keras 
from tensorflow.keras.layers  import Input , Lambda , Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np  
from glob import glob


 




In [19]:
IMAGE_SIZE = [224,224]

train_path='/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/train'
valid_path='/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/valid'

In [20]:
inception=InceptionV3(input_shape=IMAGE_SIZE +[3],weights='imagenet',include_top=False)

87916544/87910968 [==============================] - 1s 0us/step


In [21]:
for layer in inception.layers:
  layer.trainable=False

In [22]:
folders=glob('/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/train/*')

In [23]:
folders

['/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/train/Tomato___Spider_mites Two-spotted_spider_mite',
 '/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/train/Tomato___Leaf_Mold',
 '/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/train/Tomato___Tomato_mosaic_virus',
 '/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/train/Tomato___Early_blight',
 '/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/train/Tomato___Septoria_leaf_spot',
 '/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/train/Tomato___Late_blight',
 '/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/train/Tomato___Bacterial_spot',
 '/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/train/Tomato___Target_Spot',
 '/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/train/Tomato___healthy',
 '/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus']

In [24]:
x=Flatten()(inception.output)

In [25]:
prediction=Dense(len(folders),activation='softmax')(x)

model = Model(inputs=inception.input,outputs=prediction)

In [ ]:
model.summary()

In [28]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [29]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [41]:
training_set=train_datagen.flow_from_directory('/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/train',
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical')


Found 18345 images belonging to 10 classes.


In [42]:
test_set=test_datagen.flow_from_directory('/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/valid',
                                          target_size=(224,224),
                                          batch_size=32,
                                          class_mode='categorical')

Found 4585 images belonging to 10 classes.


In [44]:
r=model.fit_generator(training_set,
                      validation_data=test_set,
                      epochs=1,
                      steps_per_epoch=len(training_set),
                      validation_steps=len(test_set)
                      )

574/574 [==============================] - 370s 645ms/step - loss: 2.0560 - accuracy: 0.7898 - val_loss: 1.5808 - val_accuracy: 0.8268


In [45]:
model.save('tomato.h5')

In [46]:
y_pred=model.predict(test_set)

In [47]:
y_pred

array([[1.6965107e-27, 9.0240078e-18, 6.6573985e-23, ..., 1.7494750e-28,
        1.0692515e-16, 2.1788437e-13],
       [2.7286976e-23, 3.0769404e-12, 1.4001880e-13, ..., 2.3538785e-26,
        9.3433672e-01, 1.8885926e-23],
       [1.0000000e+00, 2.9291763e-17, 2.3004107e-23, ..., 1.5299946e-19,
        9.8536907e-29, 8.2085792e-28],
       ...,
       [5.8169037e-24, 3.5592760e-09, 1.0000000e+00, ..., 7.4114296e-26,
        1.7792267e-24, 2.5070213e-20],
       [8.2847691e-05, 7.6398784e-08, 2.3515417e-12, ..., 1.0832255e-02,
        6.7615956e-06, 6.8235992e-33],
       [1.5099301e-23, 8.3673752e-22, 1.9313632e-30, ..., 2.1843318e-32,
        1.4341856e-27, 4.1790075e-28]], dtype=float32)

In [ ]:
y_pred=np.argmax(y_pred,axis=1)
 

In [52]:
y_pred

array([5, 8, 0, ..., 2, 4, 5])

In [57]:
MODEL_PATH='/content/tomato.h5'
from tensorflow.python.keras.saving.save import load_model
model=load_model(MODEL_PATH)

In [60]:
def model_predict(img_path, model):
    print(img_path)
    img = image.load_img(img_path, target_size=(224, 224))

    # Preprocessing the image
    x = image.img_to_array(img)
    x = np.true_divide(x, 255)
    # Scaling
    x=x/255
    x = np.expand_dims(x, axis=0)
   

    # Be careful how your trained model deals with the input
    # otherwise, it won't make correct prediction!
   # x = preprocess_input(x)

    preds = model.predict(x)
    preds=np.argmax(preds, axis=1)
    if preds==0:
        preds="The Disease is Pepper__bell___Bacterial_spot"
    elif preds==1:
        preds="The Disease is Pepper__bell___healthy"
    elif preds==2:
        preds="The Disease is Potato___Early_blight"
    elif preds==3:
        preds="Te Disease is Potato___healthy"
    elif preds==4:
        preds="The Disease is Potato___Late_blight"
    elif preds==5:
        preds="The Disease is Tomato__Tomato_mosaic_virus"
    elif preds==6:
        preds="The Disease is Tomato__Tomato_YellowLeaf__Curl_Virus"
    elif preds==7:
        preds="The Disease is Tomato_Bacterial_spot"
    elif preds==8:
        preds="The Disease is Tomato_Early_blight"
    elif preds==9:
        preds="The Disease is Pepper__bell___Bacterial_spot"
    elif preds==10:
        preds="The Disease is Pepper__bell___Bacterial_spot"
    elif preds==11:
        preds="The Disease is Pepper__bell___Bacterial_spot"
    elif preds==12:
        preds="The Disease is Pepper__bell___Bacterial_spot"
    elif preds==13:
        preds="The Disease is Pepper__bell___Bacterial_spot"
    
    
    return preds
model_predict('/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/valid/Tomato___Late_blight/005a2c1f-4e15-49e4-9e5c-61dc3ecf9708___RS_Late.B 5096.JPG', model)


/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/valid/Tomato___Late_blight/005a2c1f-4e15-49e4-9e5c-61dc3ecf9708___RS_Late.B 5096.JPG


'The Disease is Tomato_Early_blight'